In [34]:
import torch
import torch.nn.functional
import typing

In [91]:
def circular_pad(
    tensor: torch.Tensor, 
    pad: tuple[tuple[int, int], tuple[int, int]]
) -> torch.Tensor:
    
    before_0, after_0 = pad[0]
    before_1, after_1 = pad[1]
    _, n, m = tensor.size()

    padded = torch.zeros(
        size=(
            1, 
            n+before_0+after_0, 
            m+before_1+after_1
        )
    )

    padded[:, before_0:-after_0, before_1:-after_1] = tensor

    padded[:, :before_0, before_1:-after_1] = tensor[:, -before_0:, :]
    padded[:, -after_0:, before_1:-after_1] = tensor[:, :after_0, :]

    padded[:, :, :before_1] = padded[:, :, -after_1-before_1:-before_1]
    padded[:, :, -after_1:] = padded[:, :, before_1:before_1+after_1]

    # padded[:, :, before_1:-after_1] = tensor[:, :after_0, :]
    

    return padded

In [92]:
arr4x4 = torch.arange(0., 16., requires_grad=True).view(1, 4, 4)
arr4x4

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]]], grad_fn=<ViewBackward0>)

In [93]:
circ_padded = circular_pad(arr4x4, ((2, 2), (2, 2)))
circ_padded

tensor([[[10., 11.,  8.,  9., 10., 11.,  8.,  9.],
         [14., 15., 12., 13., 14., 15., 12., 13.],
         [ 2.,  3.,  0.,  1.,  2.,  3.,  0.,  1.],
         [ 6.,  7.,  4.,  5.,  6.,  7.,  4.,  5.],
         [10., 11.,  8.,  9., 10., 11.,  8.,  9.],
         [14., 15., 12., 13., 14., 15., 12., 13.],
         [ 2.,  3.,  0.,  1.,  2.,  3.,  0.,  1.],
         [ 6.,  7.,  4.,  5.,  6.,  7.,  4.,  5.]]], grad_fn=<CopySlices>)

In [94]:
torch.flip(circ_padded, dims=(1, 2))

tensor([[[ 5.,  4.,  7.,  6.,  5.,  4.,  7.,  6.],
         [ 1.,  0.,  3.,  2.,  1.,  0.,  3.,  2.],
         [13., 12., 15., 14., 13., 12., 15., 14.],
         [ 9.,  8., 11., 10.,  9.,  8., 11., 10.],
         [ 5.,  4.,  7.,  6.,  5.,  4.,  7.,  6.],
         [ 1.,  0.,  3.,  2.,  1.,  0.,  3.,  2.],
         [13., 12., 15., 14., 13., 12., 15., 14.],
         [ 9.,  8., 11., 10.,  9.,  8., 11., 10.]]], grad_fn=<FlipBackward0>)

In [23]:
arr4x4 = torch.arange(0., 16., requires_grad=True).view(1, 4, 4)
arr4x4

tensor([[[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.]]], grad_fn=<ViewBackward0>)

In [33]:
mode = 'circular'
tmp = torch.nn.functional.pad(arr4x4, pad=((0, 0), (1, 1)), mode=mode)
tmp

TypeError: pad(): argument 'pad' must be tuple of ints, not tuple

In [17]:
tmp.requires_grad

True

In [18]:
tmp = torch.nn.functional.pad((tmp.T).unsqueeze(0), pad=(1, 1), mode=mode)
tmp

tensor([[[15.,  3.,  7., 11., 15.,  3.],
         [12.,  0.,  4.,  8., 12.,  0.],
         [13.,  1.,  5.,  9., 13.,  1.],
         [14.,  2.,  6., 10., 14.,  2.],
         [15.,  3.,  7., 11., 15.,  3.],
         [12.,  0.,  4.,  8., 12.,  0.]]], grad_fn=<CopySlices>)

In [72]:
torch.nn.functional.pad(arr4x4, ((1, 1)), mode='constant')
# arr4x4.take(index=0)

tensor([[ 0.,  0.,  1.,  2.,  3.,  0.],
        [ 0.,  4.,  5.,  6.,  7.,  0.],
        [ 0.,  8.,  9., 10., 11.,  0.],
        [ 0., 12., 13., 14., 15.,  0.]], grad_fn=<ConstantPadNdBackward0>)

In [73]:
# 'constant', 'reflect', 'replicate' or 'circular'


tensor([[ 3.,  0.,  1.,  2.,  3.,  0.],
        [ 7.,  4.,  5.,  6.,  7.,  4.],
        [11.,  8.,  9., 10., 11.,  8.],
        [15., 12., 13., 14., 15., 12.]], grad_fn=<SqueezeBackward1>)

In [74]:
tmp = torch.nn.functional.pad((tmp.T).unsqueeze(0), pad=(1, 1), mode=mode)
tmp
# torch.nn.functional.pad(arr4x4.take(index=1), ((1, 1)), mode='constant')

tensor([[[15.,  3.,  7., 11., 15.,  3.],
         [12.,  0.,  4.,  8., 12.,  0.],
         [13.,  1.,  5.,  9., 13.,  1.],
         [14.,  2.,  6., 10., 14.,  2.],
         [15.,  3.,  7., 11., 15.,  3.],
         [12.,  0.,  4.,  8., 12.,  0.]]], grad_fn=<CopySlices>)

In [75]:
tmp[0, 1, 1] = 1.
tmp

tensor([[[15.,  3.,  7., 11., 15.,  3.],
         [12.,  1.,  4.,  8., 12.,  0.],
         [13.,  1.,  5.,  9., 13.,  1.],
         [14.,  2.,  6., 10., 14.,  2.],
         [15.,  3.,  7., 11., 15.,  3.],
         [12.,  0.,  4.,  8., 12.,  0.]]], grad_fn=<CopySlices>)

In [77]:
arr4x4

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]], grad_fn=<ViewBackward0>)

In [76]:
# import torch
# dx = torch.zeros(size=(4, 4), requires_grad=True)
# dy = torch.zeros(size=(4, 4), requires_grad=True)

# tv_iso = torch.sum(torch.sqrt( dx**2 + dy**2 ))
# tv_iso.backward()
tmp.requires_grad

True

In [60]:
arr4x4 = torch.arange(0., 16.).view(4, 4)
arr4x4

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [62]:
derivation(arr4x4, axis=1, detach=False)

tensor([[-12., -12., -12., -12.],
        [  4.,   4.,   4.,   4.],
        [  4.,   4.,   4.,   4.],
        [  4.,   4.,   4.,   4.]])


tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [38]:
t = torch.narrow(arr4x4, dim=0, start=0, length=2)
t -= 1
t

tensor([[-1.,  0.,  1.,  2.],
        [ 3.,  4.,  5.,  6.]])

In [39]:
arr4x4

tensor([[-1.,  0.,  1.,  2.],
        [ 3.,  4.,  5.,  6.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]])

In [52]:
import torch

# Example tensor with 4 dimensions
my_tensor = arr4x4

# Index of the dimension you want to select
index = 0

# Determine the number of dimensions
num_dims = my_tensor.dim()

# Create a range of indices to select the desired dimension
indices = torch.arange(0, num_dims)

# Replace the index with the desired dimension index
indices[index] = num_dims - 1

# Use torch.index_select to select the dimension
t = torch.index_select(my_tensor, dim=indices[index], index=torch.arange(my_tensor.size(index)))

# Print the resulting tensor
print(t.shape)

torch.Size([4, 4])


In [78]:
import torch
dx = torch.zeros(size=(4, 4), requires_grad=False)
dy = torch.zeros(size=(4, 4), requires_grad=True)
d = dx + dy
d.requires_grad

True